# RAG Evaluation Challenge

1. Prepare the evaluation data set using the products pdf files. For this challenge, we can simulate with at least 10 rows.
2. Perform evaluations on Groundedness, Relevance, Coherence, Fluency, Similarity as well as the 4 Risk and Safety Metrics.
3. Display the results in Azure AI Foundry

## Load Azure Configurations

In [1]:
from dotenv import load_dotenv
import os

load_dotenv() # take environment variables from .env.

azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT")
azure_openai_embeddings_deployment = os.getenv("AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT")
azure_openai_api_version = "2024-12-01-preview"
azure_search_service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
azure_search_service_admin_key = os.getenv("AZURE_SEARCH_ADMIN_KEY")
azure_search_service_index_name = "products-index"
azure_storage_connection_string = os.getenv("AZURE_STORAGE_CONNECTION_STRING")

model_config = {
    "azure_endpoint": azure_openai_endpoint,
    "api_key": azure_openai_key,
    "azure_deployment": azure_openai_deployment,
}

azure_subscription_id = os.getenv("AZURE_SUBSCRIPTION_ID")
azure_resource_group_name = os.getenv("AZURE_RESOURCE_GROUP_NAME")
azure_project_name = os.getenv("AZURE_PROJECT_NAME")

azure_ai_project = {
    "subscription_id": azure_subscription_id,
    "resource_group_name": azure_resource_group_name,
    "project_name": azure_project_name,
}

## Load Documents

In [2]:
from langchain_community.document_loaders import DirectoryLoader

path = "Data/products/"
loader = DirectoryLoader(path, glob="**/*.pdf")
docs = loader.load()

Error loading file Data/products/product_info_17.pdf


ImportError: libGL.so.1: cannot open shared object file: No such file or directory

## Prepare the Language Model

In [ ]:
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

generator_llm = LangchainLLMWrapper(AzureChatOpenAI(
    openai_api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    azure_deployment=azure_openai_deployment,
    model=azure_openai_deployment,
    validate_base_url=False,
    api_key=azure_openai_key
))

# init the embeddings for answer_relevancy, answer_correctness and answer_similarity
generator_embeddings = LangchainEmbeddingsWrapper(AzureOpenAIEmbeddings(
    openai_api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    azure_deployment=azure_openai_embeddings_deployment,
    model=azure_openai_embeddings_deployment,
    api_key=azure_openai_key
))

## Generate the Test Set

In [ ]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

## Save just the query and ground truth to a JSONL file for evaluation

In [ ]:
import pandas as pd

df = dataset.to_pandas()

# Create DataFrame
df = dataset.to_pandas()

# Function to clean text
def clean_text(text_list):
    cleaned_text_list = []
    for text in text_list:
        # Remove the UUID (assuming it's always at the start and followed by two newlines)
        cleaned_text = text.split('\n\n', 1)[-1]
        cleaned_text_list.append(cleaned_text)
    return cleaned_text_list

# Apply the function to the 'reference_contexts' column to remove UUID at the start
df['reference_contexts'] = df['reference_contexts'].apply(clean_text)

# Save to CSV file
df.to_csv('data/output/products-ragas-testset.csv', index=False)

# Create a new DataFrame for EvalCollection
eval_collection = pd.DataFrame(columns=['query', 'response', 'context', 'ground_truth'])

# Populate the new DataFrame
eval_collection['query'] = df['user_input']
eval_collection['ground_truth'] = df['reference']
eval_collection['response'] = ''
eval_collection['context'] = ''

# Save the DataFrame as a JSONL file
eval_collection.to_json('Data/output/products-groundtruth.jsonl', orient='records', lines=True)

## Generate the Response and Context from the Language Model

In [ ]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.core.credentials import AzureKeyCredential
from openai import AzureOpenAI

# Get credential from Azure AI Search Admin key
credential = AzureKeyCredential(azure_search_service_admin_key)
search_client = SearchClient(endpoint=azure_search_service_endpoint, 
                             credential=credential, 
                             index_name=azure_search_service_index_name)

# Azure OpenAI client
openai_client = AzureOpenAI(
    # to get version: https://learn.microsoft.com/en-us/azure/ai-services/openai/api-version-deprecation
    api_version=azure_openai_api_version,
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_key)

# Provide instructions to the model
SYSTEM_PROMPT="""
You are an AI assistant that helps users learn from the information found in the source material.
Answer the query using only the sources provided below.
Use bullets if the answer has multiple points.
If the answer is longer than 3 sentences, provide a summary.
Answer ONLY with the facts listed in the list of sources below. Cite your source when you answer the question
If there isn't enough information below, say you don't know.
Do not generate answers that don't use the sources below.
Query: {query}
Sources:\n{sources}
"""
# Iterate over each row in eval_collection
for index, row in eval_collection.iterrows():
    # User Query
    query = row['query']  

    # Convert query into vector form
    vector_query = VectorizableTextQuery(text=query, 
                                        k_nearest_neighbors=50, 
                                        fields="text_vector",
                                        weight=1)

    results = search_client.search(
        query_type="semantic", 
        semantic_configuration_name='my-semantic-config',
        search_text=query,
        vector_queries= [vector_query],
        select=["title","chunk"],
        top=3,
    )

    # Use a unique separator to make the sources distinct. 
    # We chose repeated equal signs (=) followed by a newline because it's unlikely the source documents contain this sequence.
    sources_formatted = "=================\n".join([f'TITLE: {document["title"]}, CONTENT: {document["chunk"]}' for document in results])

    # Update the context in the DataFrame
    eval_collection.at[index, 'context'] = sources_formatted

    response = openai_client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": SYSTEM_PROMPT.format(query=query, sources=sources_formatted)
            }
        ],
        model=azure_openai_deployment
    )

    # Update the response in the DataFrame
    eval_collection.at[index, 'response'] = response.choices[0].message.content

# Save the updated DataFrame as a JSONL file
eval_collection.to_json('Data/output/products-evalset.jsonl', orient='records', lines=True)

# Print success message
print("EvalCollection has been saved to products-evalet.jsonl")


## Run evaluation and view in Azure AI Foundry

In [ ]:
from azure.ai.evaluation import evaluate
from azure.ai.evaluation import GroundednessEvaluator, RetrievalEvaluator, RelevanceEvaluator, CoherenceEvaluator, FluencyEvaluator, SimilarityEvaluator, F1ScoreEvaluator
from azure.ai.evaluation import RougeScoreEvaluator, RougeType
from azure.ai.evaluation import BleuScoreEvaluator
from azure.ai.evaluation import MeteorScoreEvaluator
from azure.ai.evaluation import GleuScoreEvaluator
from azure.ai.evaluation import ViolenceEvaluator, HateUnfairnessEvaluator, SelfHarmEvaluator,SexualEvaluator
from azure.identity import DefaultAzureCredential
import pandas as pd

groundedness_eval = GroundednessEvaluator(model_config)
retrieval_eval = RetrievalEvaluator(model_config)
relevance_eval = RelevanceEvaluator(model_config)
coherence_eval = CoherenceEvaluator(model_config)
fluency_eval = FluencyEvaluator(model_config)
similarity_eval = SimilarityEvaluator(model_config)
f1_eval = F1ScoreEvaluator()
rouge_eval = RougeScoreEvaluator(rouge_type=RougeType.ROUGE_1)
bleu_eval = BleuScoreEvaluator()
meteor_eval = MeteorScoreEvaluator(
    alpha=0.9,
    beta=3.0,
    gamma=0.5
)
gleu_eval = GleuScoreEvaluator()
violence_eval = ViolenceEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
hateunfairness_eval = HateUnfairnessEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
selfharm_eval = SelfHarmEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())
sexual_eval = SexualEvaluator(azure_ai_project=azure_ai_project, credential=DefaultAzureCredential())

path = "Data/output/product-eval.jsonl"

result = evaluate(
    data=path, # provide your data here
    evaluators={
        "groundedness": groundedness_eval,
        "retrieval": retrieval_eval,
        "relevance": relevance_eval,
        "coherence": coherence_eval,
        "fluency": fluency_eval,
        "similarity": similarity_eval,
        "f1_score": f1_eval,
        "rouge_score": rouge_eval,
        "bleu_score": bleu_eval,
        "meteor_score": meteor_eval,
        "gleu_score": gleu_eval,
        "violence_score": violence_eval,
        "hateunfairness_score": hateunfairness_eval,
        "selfharm_score": selfharm_eval,
        "sexual_score": sexual_eval 
    },
    # column mapping
    evaluator_config={
        "default": {
            "query": "${data.query}",
            "response": "${data.response}",
            "context": "${data.context}",
            "ground_truth": "${data.ground_truth}"
        }
    },
    azure_ai_project = azure_ai_project
)
